# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [9]:
file_path = 'C:\\Users\\David Santos\\Desktop\\Labs IronHack\\Prontos\\lab-dataframe-calculations\\your-code\\Orders.csv'

orders = pd.read_csv(file_path)

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [10]:
# Sub Problem 1: 

customer_agg = orders.groupby('CustomerID')['amount_spent'].sum().reset_index()

In [11]:
# Sub Problem 2:

# quantile range (between 25th and 75th percentiles)
lower_quantile = customer_agg['amount_spent'].quantile(0.25)
upper_quantile = customer_agg['amount_spent'].quantile(0.75)

# customers within the specified quantile range
selected_customers = customer_agg[(customer_agg['amount_spent'] >= lower_quantile) & (customer_agg['amount_spent'] <= upper_quantile)]

In [12]:
# Sub Problem 3:

selected_customers_copy = selected_customers.copy()

# function to label customers
def label_customer(row):
    if row['amount_spent'] >= upper_quantile:
        return "VIP"
    elif row['amount_spent'] >= lower_quantile:
        return "Preferred"
    else:
        return "Regular"

# labeling function to the copied DataFrame
selected_customers_copy['CustomerLabel'] = selected_customers_copy.apply(lambda row: label_customer(row), axis=1)

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [16]:

merged_data = pd.merge(selected_customers_copy, orders[['CustomerID', 'Country']], on='CustomerID', how='left')


vip_customers = merged_data[merged_data['CustomerLabel'] == 'VIP']

vip_count_by_country = vip_customers.groupby('Country')['CustomerID'].count().reset_index()

most_vip_country = vip_count_by_country[vip_count_by_country['CustomerID'] == vip_count_by_country['CustomerID'].max()]


print("Country with the most VIP Customers:")
print(most_vip_country)

Country with the most VIP Customers:
Empty DataFrame
Columns: [Country, CustomerID]
Index: []


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [21]:
selected_customers_copy = selected_customers.copy()


def label_customer(row):
    if row['amount_spent'] >= upper_quantile:
        return "VIP"
    elif row['amount_spent'] >= lower_quantile:
        return "Preferred"
    else:
        return "Regular"

selected_customers_copy['CustomerLabel'] = selected_customers_copy.apply(lambda row: label_customer(row), axis=1)

merged_data = pd.merge(selected_customers_copy, orders[['CustomerID', 'Country']], on='CustomerID', how='left')

vip_preferred_customers = merged_data[(merged_data['CustomerLabel'] == 'VIP') | (merged_data['CustomerLabel'] == 'Preferred')]

vip_preferred_count_by_country = vip_preferred_customers.groupby('Country')['CustomerID'].count().reset_index()


most_vip_preferred_country = vip_preferred_count_by_country[vip_preferred_count_by_country['CustomerID'] == vip_preferred_count_by_country['CustomerID'].max()]


print("Country with the most VIP+Preferred Customers:")
print(most_vip_preferred_country)

Country with the most VIP+Preferred Customers:
           Country  CustomerID
29  United Kingdom      114660
